# 13.1  Class là instance của type

In [1]:
type(list),type(type)

(type, type)

In [2]:
class C:
    pass
I=C()
type(C),type(I)

(type, __main__.C)

In [3]:
C.__class__,I.__class__

(type, __main__.C)

**Cú pháp khai báo lớp:** 

`class=type(classname,superclass,attributedict)`

In [4]:
Spam=type('Spam',(),{'data':1})
Spam

__main__.Spam

In [5]:
class Spam:
    data=1
    pass
Spam

__main__.Spam

In [6]:
x = type('Spam', (), {'data': 1, 'meth': (lambda x, y: x.data + y)})
i = x()
x, i

(__main__.Spam, <__main__.Spam at 0x3408b30>)

In [7]:
i.data, i.meth(2)

(1, 3)

In [8]:
x.__bases__

(object,)

In [9]:
 [(a, v) for (a, v) in x.__dict__.items() if not a.startswith('__')]

[('data', 1), ('meth', <function __main__.<lambda>(x, y)>)]

**Cú pháp khai báo metaclasses:**

`class Spam(metaclass=Meta)`

In [10]:
class Meta(type):
    def __new__(meta, classname, supers, classdict):
        # Run by inherited type.__call__
        return type.__new__(meta, classname, supers, classdict)

In [11]:
class MetaOne(type):
    def __new__(meta, classname, supers, classdict):
        print('In MetaOne.new:', meta, classname, supers, classdict, sep='\n...')
        return type.__new__(meta, classname, supers, classdict)
    
class Eggs:
    pass

print('making class')

class Spam(Eggs, metaclass=MetaOne): # Inherits from Eggs, instance of MetaOne
    data = 1 # Class data attribute
    def meth(self, arg): # Class method attribute
        return self.data + arg
    
print('making instance')
X = Spam()
print('data:', X.data, X.meth(2))

making class
In MetaOne.new:
...<class '__main__.MetaOne'>
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x0343C030>}
making instance
data: 1 3


## Customize costructor and init

In [12]:
class MetaTwo(type):
    def __new__(meta, classname, supers, classdict):
        print('In MetaTwo.new: ', classname, supers, classdict, sep='\n...')
        return type.__new__(meta, classname, supers, classdict)
    def __init__(Class, classname, supers, classdict):
        print('In MetaTwo.init:', classname, supers, classdict, sep='\n...')
        print('...init class object:', list(Class.__dict__.keys()))
        
class Eggs:
    pass
print('making class')

class Spam(Eggs, metaclass=MetaTwo): # Inherits from Eggs, instance of MetaTwo
    data = 1 # Class data attribute
    def meth(self, arg): # Class method attribute
        return self.data + arg
    
print('making instance')
X = Spam()
print('data:', X.data, X.meth(2))

making class
In MetaTwo.new: 
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x0343C348>}
In MetaTwo.init:
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x0343C348>}
...init class object: ['__module__', 'data', 'meth', '__doc__']
making instance
data: 1 3


In [13]:
def MetaFunc(classname, supers, classdict):
    print('In MetaFunc: ', classname, supers, classdict, sep='\n...')
    return type(classname, supers, classdict)

class Eggs:
    pass

print('making class')
class Spam(Eggs, metaclass=MetaFunc): # Run simple function at end
    data = 1 # Function returns class
    def meth(self, arg):
        return self.data + arg
    
print('making instance')
X = Spam()
print('data:', X.data, X.meth(2))

making class
In MetaFunc: 
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x0343C930>}
making instance
data: 1 3


In [14]:
class MetaObj:
    def __call__(self, classname, supers, classdict):
        print('In MetaObj.call: ', classname, supers, classdict, sep='\n...')
        Class = self.__New__(classname, supers, classdict)
        self.__Init__(Class, classname, supers, classdict)
        return Class
    
    def __New__(self, classname, supers, classdict):
        print('In MetaObj.new: ', classname, supers, classdict, sep='\n...')
        return type(classname, supers, classdict)
    
    def __Init__(self, Class, classname, supers, classdict):
        print('In MetaObj.init:', classname, supers, classdict, sep='\n...')
        print('...init class object:', list(Class.__dict__.keys()))
        
class Eggs:
    pass

print('making class')
class Spam(Eggs, metaclass=MetaObj()): # MetaObj is normal class instance
    data = 1 # Called at end of statement
    def meth(self, arg):
        return self.data + arg
    
print('making instance')
X = Spam()
print('data:', X.data, X.meth(2))

making class
In MetaObj.call: 
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x0343C348>}
In MetaObj.new: 
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x0343C348>}
In MetaObj.init:
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x0343C348>}
...init class object: ['__module__', 'data', 'meth', '__doc__']
making instance
data: 1 3


In [15]:
class SuperMetaObj:
    def __call__(self, classname, supers, classdict):
        print('In SuperMetaObj.call: ', classname, supers, classdict, sep='\n...')
        Class = self.__New__(classname, supers, classdict)
        self.__Init__(Class, classname, supers, classdict)
        return Class
    
class SubMetaObj(SuperMetaObj):
    def __New__(self, classname, supers, classdict):
        print('In SubMetaObj.new: ', classname, supers, classdict, sep='\n...')
        return type(classname, supers, classdict)
    
    def __Init__(self, Class, classname, supers, classdict):
        print('In SubMetaObj.init:', classname, supers, classdict, sep='\n...')
        print('...init class object:', list(Class.__dict__.keys()))
        
class Spam(Eggs, metaclass=SubMetaObj()):
    data = 1 # Called at end of statement
    def meth(self, arg):
        return self.data + arg
    
print('making instance')
X = Spam()
print('data:', X.data, X.meth(2))

In SuperMetaObj.call: 
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x0343CC00>}
In SubMetaObj.new: 
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x0343CC00>}
In SubMetaObj.init:
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x0343CC00>}
...init class object: ['__module__', 'data', 'meth', '__doc__']
making instance
data: 1 3


In [16]:
class SuperMeta(type):
    def __call__(meta, classname, supers, classdict):
        print('In SuperMeta.call: ', classname, supers, classdict, sep='\n...')
        return type.__call__(meta, classname, supers, classdict)
    
    def __init__(Class, classname, supers, classdict):
        print('In SuperMeta init:', classname, supers, classdict, sep='\n...')
        print('...init class object:', list(Class.__dict__.keys()))
        
print('making metaclass')
class SubMeta(type, metaclass=SuperMeta):
    def __new__(meta, classname, supers, classdict):
        print('In SubMeta.new: ', classname, supers, classdict, sep='\n...')
        return type.__new__(meta, classname, supers, classdict)
    
    def __init__(Class, classname, supers, classdict):
        print('In SubMeta init:', classname, supers, classdict, sep='\n...')
        print('...init class object:', list(Class.__dict__.keys()))
        
class Eggs:
    pass

print('making class')
class Spam(Eggs, metaclass=SubMeta): # Invoke SubMeta, via SuperMeta.__call__
    data = 1
    def meth(self, arg):
        return self.data + arg
    
print('making instance')
X = Spam()
print('data:', X.data, X.meth(2))

making metaclass
In SuperMeta init:
...SubMeta
...(<class 'type'>,)
...{'__module__': '__main__', '__qualname__': 'SubMeta', '__new__': <function SubMeta.__new__ at 0x0343C420>, '__init__': <function SubMeta.__init__ at 0x0343C270>}
...init class object: ['__module__', '__new__', '__init__', '__doc__']
making class
In SuperMeta.call: 
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x0343CFA8>}
In SubMeta.new: 
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x0343CFA8>}
In SubMeta init:
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x0343CFA8>}
...init class object: ['__module__', 'data', 'meth', '__doc__']
making instance
data: 1 3


In [17]:
class SuperMeta(type):
    def __call__(meta, classname, supers, classdict): # By name, not built-in
        print('In SuperMeta.call:', classname)
        return type.__call__(meta, classname, supers, classdict)
    
class SubMeta(SuperMeta): # Created by type default
    def __init__(Class, classname, supers, classdict): # Overrides type.__init__
        print('In SubMeta init:', classname)
        
print(SubMeta.__class__)
print([n.__name__ for n in SubMeta.__mro__])
print()
print(SubMeta.__call__) # Not a data descriptor if found by name
print()
SubMeta.__call__(SubMeta, 'xxx', (), {}) # Explicit calls work: class inheritance
print()
SubMeta('yyy', (), {})

<class 'type'>
['SubMeta', 'SuperMeta', 'type', 'object']

<function SuperMeta.__call__ at 0x03406DB0>

In SuperMeta.call: xxx
In SubMeta init: xxx

In SubMeta init: yyy


__main__.yyy

## Inheritance and Instance

In [18]:
class MetaOne(type):
    def __new__(meta, classname, supers, classdict): # Redefine
        print('In MetaOne.new:', classname)
        return type.__new__(meta, classname, supers, classdict)
    
    def toast(self):
        return 'toast'
    
class Super(metaclass=MetaOne): # Metaclass inherited by subs too
    def spam(self): # MetaOne run twice for two classes
        return 'spam'
    
class Sub(Super): # Superclass: inheritance versus instance
    def eggs(self): # Classes inherit from superclasses
        return 'eggs' # But not from metaclasse

In MetaOne.new: Super
In MetaOne.new: Sub


In [19]:
X = Sub()
X.eggs()

'eggs'

In [20]:
X.spam()

'spam'

Ta thử truy cập vào phương thức của lớp MetaOne.
```python
X.toast() 
```
`Output: AttributeError: 'Sub' object has no attribute 'toast'`

In [21]:
Sub.toast

<bound method MetaOne.toast of <class '__main__.Sub'>>

In [22]:
 Sub.spam

<function __main__.Super.spam(self)>

In [23]:
X.spam

<bound method Super.spam of <__main__.Sub object at 0x0343BCB0>>

In [24]:
class A(type): attr = 1
class B(metaclass=A): pass
I = B() # I inherits from class but not meta!
B.attr  # Error with I.attr

1

In [25]:
'attr' in B.__dict__, 'attr' in A.__dict__

(False, True)

In [26]:
class A: attr = 1
class B(A): pass # I inherits from class and super
I = B()
B.attr

1

In [27]:
I.attr

1

In [28]:
'attr' in B.__dict__, 'attr' in A.__dict__

(False, True)

In [29]:
class M(type): attr = 1
class A: attr = 2
class B(A, metaclass=M): pass
I = B()
B.attr, I.attr

(2, 2)

In [30]:
'attr' in B.__dict__, 'attr' in A.__dict__, 'attr' in M.__dict__

(False, True, True)

In [31]:
class M(type): attr = 1
class A: attr = 2
class B(A): pass
class C(B, metaclass=M): pass # Super two levels above meta: still wins
I = C()
I.attr, C.attr

(2, 2)

In [32]:
[x.__name__ for x in C.__mro__]

['C', 'B', 'A', 'object']

In [33]:
I.__class__ # Followed by inheritance: instance's clas

__main__.C

In [34]:
 C.__bases__

(__main__.B,)

In [35]:
C.__class__

__main__.M

In [36]:
C.__class__.attr # Another way to get to metaclass attribute

1

In [37]:
class M1(type): attr1 = 1 # Metaclass inheritance tree
class M2(M1): attr2 = 2

In [38]:
class C1: attr3 = 3 # Superclass inheritance tree
class C2(C1,metaclass=M2): attr4 = 4

In [39]:
I = C2() # I gets __class__ but not others
I.attr3, I.attr4

(3, 4)

In [40]:
C2.attr1, C2.attr2, C2.attr3, C2.attr4

(1, 2, 3, 4)

In [41]:
 M2.attr1, M2.attr2

(1, 2)

In [42]:
I.__class__ # Links followed at instance with no __base

__main__.C2

In [43]:
C2.__bases__

(__main__.C1,)

In [44]:
C2.__class__ # Links followed at class after __base

__main__.M2

In [45]:
M2.__bases__

(__main__.M1,)

In [46]:
I.__class__.attr1 # Route inheritance to the class's meta tree

1

In [47]:
M2.__class__

type

In [48]:
[x.__name__ for x in C2.__mro__]

['C2', 'C1', 'object']

In [49]:
[x.__name__ for x in M2.__mro__]

['M2', 'M1', 'type', 'object']

In [50]:
class C: pass # Inheritance special case #1...
I = C() # Class data descriptors have precedence
I.__class__, I.__dict__

(__main__.C, {})

In [51]:
I.__dict__['name'] = 'bob' # Dynamic data in the instance
I.__dict__['__class__'] = 'spam' # Assign keys, not attributes
I.__dict__['__dict__'] = {}

In [52]:
I.name

'bob'

In [53]:
I.__class__, I.__dict__

(__main__.C, {'name': 'bob', '__class__': 'spam', '__dict__': {}})

In [54]:
class D:
    def __get__(self, instance, owner): print('__get__')
    def __set__(self, instance, value): print('__set__')

In [55]:
class C: d = D() # Data descriptor attribute
I = C()
I.d

__get__


In [56]:
I.d = 1

__set__


In [57]:
I.__dict__['d'] = 'spam'
I.d

__get__


In [58]:
class D:
    def __get__(self, instance, owner): print('__get__')
class C: d = D()
I = C()
I.d

__get__


In [59]:
I.__dict__['d'] = 'spam' # Hides class names per normal inheritance rules
I.d

'spam'

In [60]:
class C: # Inheritance special case #2...
    attr = 1 # Built-ins skip a step
    def __str__(self): return('class')
I = C()
I.__str__(), str(I)

('class', 'class')

In [61]:
I.__str__ = lambda: 'instance'
I.__str__(), str(I)

('instance', 'class')

In [62]:
I.attr # Asymmetric with normal or explicit name

1

In [63]:
I.attr = 2; I.attr

2

In [64]:
class D(type):
    def __str__(self): return('D class')
class C(D):
    pass
C.__str__(C), str(C)

('D class', "<class '__main__.C'>")

In [65]:
class C(D):
    def __str__(self): return('C class')
C.__str__(C), str(C)

('C class', "<class '__main__.C'>")

In [66]:
class C(metaclass=D):
    def __str__(self): return('C class')
C.__str__(C), str(C)

('C class', 'D class')

In [67]:
class C(metaclass=D):
    pass
C.__str__(C), str(C)

("<class '__main__.C'>", 'D class')

In [68]:
C.__str__

<slot wrapper '__str__' of 'object' objects>

In [69]:
for k in (C, C.__class__, type): print([x.__name__ for x in k.__mro__])

['C', 'object']
['D', 'type', 'object']
['type', 'object']


# 13.1 Metaclass method

In [70]:
class A(type):
    def x(cls): print('ax', cls) # A metaclass (instances=classes)
    def y(cls): print('ay', cls) # y is overridden by instance B

class B(metaclass=A):
    def y(self): print('by', self) # A normal class (normal instances)
    def z(self): print('bz', self) # Namespace dict holds y and z

B.x # x acquired from metaclas

<bound method A.x of <class '__main__.B'>>

In [71]:
B.y

<function __main__.B.y(self)>

In [72]:
B.z

<function __main__.B.z(self)>

In [73]:
B.x()

ax <class '__main__.B'>


In [74]:
I = B() # Instance method calls: get inst
I.y()

by <__main__.B object at 0x03462D50>


In [75]:
I.z()

bz <__main__.B object at 0x03462D50>


In [76]:
class A(type):
    def a(cls): # Metaclass method: gets class
        cls.x = cls.y + cls.z
        
class B(metaclass=A):
    y, z = 11, 22
    @classmethod # Class method: gets class
    def b(cls):
        return cls.x        

In [77]:
B.a() # Call metaclass method; visible to class only
B.x

33

In [78]:
I = B()
I.x, I.y, I.z

(33, 11, 22)

In [79]:
I.b()

33

In [80]:
class A(type):
    def __getitem__(cls, i): # Meta method for processing classes:
        return cls.data[i]
    
class B(metaclass=A): # Data descriptors in meta used first
    data = 'spam'

B[0]  # Error I[0]

's'

In [81]:
B.__getitem__

<bound method A.__getitem__ of <class '__main__.B'>>

In [82]:
I = B()
I.data, B.data

('spam', 'spam')

In [83]:
class A(type):
    def __getattr__(cls, name): # Acquired by class B getitem
        return getattr(cls.data, name) # But not run same by built-in
    
class B(metaclass=A):
    
    data = 'spam'

In [84]:
B.upper()   

'SPAM'

In [85]:
B.upper

<function str.upper()>

In [86]:
B.__getattr__

<bound method A.__getattr__ of <class '__main__.B'>>

In [87]:
B.data = [1, 2, 3]
B.append(4) # Explicit normal names routed to meta's getattr
B.data

[1, 2, 3, 4]

In [88]:
B.__getitem__(0) # Error B[0]

1

In [89]:
class Client1:
    def __init__(self, value):
        self.value = value
    def spam(self):
        return self.value * 2
    
class Client2:
    value = 'ni?'
    
def eggsfunc(obj):
    return obj.value * 4

def hamfunc(obj, value):
    return value + 'ham'

Client1.eggs = eggsfunc
Client1.ham = hamfunc
Client2.eggs = eggsfunc
Client2.ham = hamfunc

X = Client1('Ni!')
print(X.spam())
print(X.eggs())
print(X.ham('bacon'))
Y = Client2()
print(Y.eggs())
print(Y.ham('bacon'))

Ni!Ni!
Ni!Ni!Ni!Ni!
baconham
ni?ni?ni?ni?
baconham


In [90]:
def eggsfunc(obj):
    return obj.value * 4

def hamfunc(obj, value):
    return value + 'ham'

class Extender(type):
    def __new__(meta, classname, supers, classdict):
        classdict['eggs'] = eggsfunc
        classdict['ham'] = hamfunc
        return type.__new__(meta, classname, supers, classdict)
    
class Client1(metaclass=Extender):
    def __init__(self, value):
        self.value = value
        
    def spam(self):
        return self.value * 2
    
class Client2(metaclass=Extender):
    value = 'ni?'
    
X = Client1('Ni!')
print(X.spam())
print(X.eggs())
print(X.ham('bacon'))
Y = Client2()
print(Y.eggs())
print(Y.ham('bacon'))

Ni!Ni!
Ni!Ni!Ni!Ni!
baconham
ni?ni?ni?ni?
baconham


In [91]:
class MetaExtend(type):
    def __new__(meta, classname, supers, classdict):
        if sometest():
            classdict['eggs'] = eggsfunc1
        else:
            classdict['eggs'] = eggsfunc2
        if someothertest():
            classdict['ham'] = hamfunc
        else:
            classdict['ham'] = lambda *args: 'Not supported'
        return type.__new__(meta, classname, supers, classdict)

# 13.3 Metaclass với Decorator

In [92]:
def eggsfunc(obj):
    return obj.value * 4

def hamfunc(obj, value):
    return value + 'ham'

def Extender(aClass):
    aClass.eggs = eggsfunc # Manages class, not instance
    aClass.ham = hamfunc # Equiv to metaclass __init__
    return aClass

@Extender
class Client1: # Client1 = Extender(Client1)
    def __init__(self, value): # Rebound at end of class stmt
        self.value = value
    def spam(self):
        return self.value * 2
    
@Extender
class Client2:
    value = 'ni?'
    
X = Client1('Ni!') # X is a Client1 instance
print(X.spam())
print(X.eggs())
print(X.ham('bacon'))
Y = Client2()
print(Y.eggs())
print(Y.ham('bacon'))

Ni!Ni!
Ni!Ni!Ni!Ni!
baconham
ni?ni?ni?ni?
baconham


In [93]:
def Tracer(aClass): # On @ decorator
    class Wrapper:
        def __init__(self, *args, **kargs): # On instance creation
            self.wrapped = aClass(*args, **kargs) # Use enclosing scope name
        def __getattr__(self, attrname):
            print('Trace:', attrname) # Catches all but .wrapped
            return getattr(self.wrapped, attrname) # Delegate to wrapped object
    return Wrapper

@Tracer
class Person: # Person = Tracer(Person)
    def __init__(self, name, hours, rate): # Wrapper remembers Person
        self.name = name
        self.hours = hours
        self.rate = rate # In-method fetch not traced
    def pay(self):
        return self.hours * self.rate
    
bob = Person('Bob', 40, 50) # bob is really a Wrapper
print(bob.name) # Wrapper embeds a Person
print(bob.pay()) # Triggers __getattr_

Trace: name
Bob
Trace: pay
2000


In [94]:
def Tracer(classname, supers, classdict): # On class creation call
    aClass = type(classname, supers, classdict) # Make client class
    
    class Wrapper:
        def __init__(self, *args, **kargs): # On instance creation
            self.wrapped = aClass(*args, **kargs)
            
        def __getattr__(self, attrname):
            print('Trace:', attrname) # Catches all but .wrapped
            return getattr(self.wrapped, attrname) # Delegate to wrapped object
    return Wrapper

class Person(metaclass=Tracer): # Make Person with Tracer
    def __init__(self, name, hours, rate): # Wrapper remembers Person
        self.name = name
        self.hours = hours
        self.rate = rate # In-method fetch not traced
        
    def pay(self):
        return self.hours * self.rate
    
bob = Person('Bob', 40, 50) # bob is really a Wrapper
print(bob.name) # Wrapper embeds a Person
print(bob.pay()) # Triggers __getatt

Trace: name
Bob
Trace: pay
2000


In [95]:
class Metaclass(type):
    def __new__(meta, clsname, supers, attrdict):
        print('In M.__new__:')
        print([clsname, supers, list(attrdict.keys())])
        return type.__new__(meta, clsname, supers, attrdict)
    
def decorator(cls):
    return Metaclass(cls.__name__, cls.__bases__, dict(cls.__dict__))


In [96]:
class A:
    x = 1
    
@decorator
class B(A):
    y = 2
    def m(self): return self.x + self.y

In M.__new__:
['B', (<class '__main__.A'>,), ['__module__', 'y', 'm', '__doc__']]


In [97]:
 B.x, B.y

(1, 2)

In [98]:
I = B()
I.x, I.y, I.m()

(1, 2, 3)

In [99]:
def func(name, supers, attrs):
    return 'spam'

class C(metaclass=func): # A class whose metaclass makes it a string!
    attr = 'huh?

Object `huh` not found.


In [ ]:
    attr = 'huh

In [100]:
C, C.upper()

('spam', 'SPAM')

In [101]:
def func(cls):
    return 'spam'

In [102]:
@func
class C: # A class whose decorator makes it a string!
    attr = 'huh?'

In [103]:
C, C.upper()

('spam', 'SPAM')

In [104]:
import time
def tracer(func): # Use function, not class with __call__
    calls = 0 # Else self is decorator instance only
    
    def onCall(*args, **kwargs):
        nonlocal calls
        calls += 1
        print('call %s to %s' % (calls, func.__name__))
        return func(*args, **kwargs)
    return onCall

def timer(label='', trace=True): # On decorator args: retain args
    def onDecorator(func): # On @: retain decorated func
        def onCall(*args, **kargs): # On calls: call original
            start = time.clock() # State is scopes + func attr
            result = func(*args, **kargs)
            elapsed = time.clock() - start
            onCall.alltime += elapsed
            
            if trace:
                format = '%s%s: %.5f, %.5f'
                values = (label, func.__name__, elapsed, onCall.alltime)
                print(format % values)
                return result
        onCall.alltime = 0
        return onCall
    return onDecorator

In [105]:
class Person:

    @tracer
    def __init__(self, name, pay):
        self.name = name
        self.pay = pay
        
    @tracer
    def giveRaise(self, percent): # giveRaise = tracer(giverRaise)
        self.pay *= (1.0 + percent) # onCall remembers giveRaise
        
    @tracer
    def lastName(self): # lastName = tracer(lastName)
        return self.name.split()[-1]
    
bob = Person('Bob Smith', 50000)
sue = Person('Sue Jones', 100000)
print(bob.name, sue.name)
sue.giveRaise(.10) # Runs onCall(sue, .10)
print('%.2f' % sue.pay)
print(bob.lastName(), sue.lastName())

call 1 to __init__
call 2 to __init__
Bob Smith Sue Jones
call 1 to giveRaise
110000.00
call 1 to lastName
call 2 to lastName
Smith Jones


In [106]:
from types import FunctionType

class MetaTrace(type):
    def __new__(meta, classname, supers, classdict):
        for attr, attrval in classdict.items():
            if type(attrval) is FunctionType: # Method?
                classdict[attr] = tracer(attrval) # Decorate it
        return type.__new__(meta, classname, supers, classdict) # Make class
    
class Person(metaclass=MetaTrace):
    def __init__(self, name, pay):
        self.name = name
        self.pay = pay
    def giveRaise(self, percent):
        self.pay *= (1.0 + percent)
    def lastName(self):
        return self.name.split()[-1]
    
bob = Person('Bob Smith', 50000)
sue = Person('Sue Jones', 100000)
print(bob.name, sue.name)
sue.giveRaise(.10)
print('%.2f' % sue.pay)
print(bob.lastName(), sue.lastName())

call 1 to __init__
call 2 to __init__
Bob Smith Sue Jones
call 1 to giveRaise
110000.00
call 1 to lastName
call 2 to lastName
Smith Jones


# Bài tập Excercise
## A. Quiz
**Câu 1:** Metaclass là gì?

**Câu 2:** Làm thế nào để bạn khai báo metaclass của một lớp?

**Câu 3:** Làm cách nào để decorator class giống với meta class cho quản lý lớp?

**Câu 4:** Làm cách nào để decorator class chồng lên metaclass để quản lý các instance?

**Câu 5:** 
## B. Coding


---
# <span style= 'color:blue'> Đáp án </span>

**1.** Metaclass là một lớp được sử dụng để tạo một lớp. Các lớp kiểu mới bình thường là các instance của lớp kiểu theo mặc định. Metaclasses thường là lớp con của lớp type,
xác định lại các phương thức giao thức tạo lớp để tùy chỉnh lớp
lệnh gọi tạo được đưa ra ở cuối một câu lệnh lớp; người ta thường xác định lại
phương thức \_\_new\_\_ và \_\_init\_\_ để thay đổi vào giao thức tạo lớp.

**2.** Trong Python 3.X sử dụng đối số từ khóa trong dòng tiêu đề lớp: class C (metaclass = M). 

**3.** Bởi vì cả hai đều được tự động kích hoạt ở cuối câu lệnh lớp, lớp
decorator và metaclasses đều có thể được sử dụng để quản lý các lớp. Decorator rebind
tên lớp cho kết quả của một tệp có thể gọi và tạo lớp định tuyến metaclasses thông qua
có thể gọi được, nhưng cả hai móc đều có thể được sử dụng cho các mục đích tương tự. Để quản lý các lớp học, decorator chỉ đơn giản là tăng thêm và trả về các đối tượng lớp ban đầu. Metaclasses tăng thêm một đẳng cấp sau khi họ tạo ra nó. Decorator có thể có một chút bất lợi trong việc này nếu một lớp mới phải được xác định, bởi vì lớp ban đầu đã được
tạo.

**4.** Bởi vì cả hai đều được tự động kích hoạt ở cuối câu lệnh lớp, chúng ta có thể
sử dụng cả trình decorator class và metaclass để quản lý các instance của lớp, bằng cách chèn
một đối tượng trình bao bọc (proxy) để bắt các cuộc gọi tạo phiên bản. Decorator có thể kết hợp lại tên lớp cho một chạy có thể gọi được khi tạo cá thể giữ lại lớp ban đầu
sự vật. Metaclasses cũng có thể làm được điều tương tự, nhưng có thể có một chút bất lợi trong việc này vì chúng cũng phải tạo đối tượng lớp.